In [ ]:
# install.packages(c("gplots","plotrix","clue","reshape","e1071"))
source("support/util.R")

# Load the data

In [ ]:
## load the document-term matrix
Ptilde <- as.matrix( read.csv("data/news.csv") )
P <- Ptilde/sum(Ptilde) ## replace word frequencies by "probabilities"

In [ ]:
## the data is sorted by newsgroup; this is not exploited/known to NMF
## here we truncate values >0.0001 for improved visiblity
ggplotm(pmin(P,0.0001), format="", show.axis=FALSE, mid="black")
# showcol(pmin(P,0.0001)) # alternative, faster but less flexible

# 1 Topic Modelling with NMF

In [ ]:
## run NMF using GKL
r <- 4
lr.gkl <- lee01.gkl(P, r, reps=5)

In [ ]:
## print results
with(lr.gkl,
     for (k in 1:nrow(R)) {
         print(rev(sort(R[k,]))[1:10])
         cat(strrep('-',80), "\n")
     })

In [ ]:
## look at the reconstruction
Phat <- lr.gkl$L %*% lr.gkl$R
summary(as.vector(Phat))
ggplotm(pmin(Phat, 0.0001), format="", show.axis=FALSE, mid="black")

In [ ]:
## also compute SVD
P.svd <- svd(P)
rownames(P.svd$v) <- colnames(P)

In [ ]:
## print results
with(P.svd,
     for (k in 1:r) {
         y=order(abs(v[,k]))
         print(rev(v[y,k])[1:10])
         cat(strrep('-',80), "\n")
     })

In [ ]:
## svd reconstruction
Phat <- P.svd$u[,1:4] %*% diag(P.svd$d[1:4]) %*% t(P.svd$v[,1:4])
summary(as.vector(Phat))
ggplotm(pmax(pmin(Phat,0.0001),-0.0001), format="", show.axis=FALSE, mid="black")

In [ ]:
## look at the decomposition (here: Rtilde and V)
ggplotm(lr.gkl$R, format="", show.axis=FALSE, mid="black") # enlarge plot or subselect columns to see something
ggplotm(lr.gkl$R[,1:30], format="", rotate.labels=TRUE, mid="black") # first 30 columns

ggplotm(t(P.svd$v[,1:4]), format="", show.axis=FALSE, mid="black")
ggplotm(t(P.svd$v[1:30,1:4]), format="", rotate.labels=TRUE, mid="black") # first 30 columns

## Rank 2 NMF

In [ ]:
## run NMF using GKL with rank = 2
r <- 2
lr.gkl.r2 <- lee01.gkl(P, r, reps=5)

In [ ]:
## print results
with(lr.gkl.r2,
     for (k in 1:nrow(R)) {
         print(rev(sort(R[k,]))[1:10])
         cat(strrep('-',80), "\n")
     })

In [ ]:
## look at the reconstruction
Phat.r2 <- lr.gkl.r2$L %*% lr.gkl.r2$R
summary(as.vector(Phat.r2))
ggplotm(pmin(Phat.r2, 0.0001), format="", show.axis=FALSE, mid="black")

## Rank 8 NMF

In [ ]:
## run NMF using GKL with rank = 8
r <- 8
lr.gkl.r8 <- lee01.gkl(P, r, reps=5)

In [ ]:
## print results
with(lr.gkl.r8,
     for (k in 1:nrow(R)) {
         print(rev(sort(R[k,]))[1:10])
         cat(strrep('-',80), "\n")
     })

In [ ]:
## look at the reconstruction
Phat.r8 <- lr.gkl.r8$L %*% lr.gkl.r8$R
summary(as.vector(Phat.r8))
ggplotm(pmin(Phat.r8, 0.0001), format="", show.axis=FALSE, mid="black")

## Gaussian NMF

In [ ]:
r <- 4
lr.gnmf <- lee01.gnmf(P, r, reps=5)

In [ ]:
## print results
with(lr.gnmf,
     for (k in 1:nrow(R)) {
         print(rev(sort(R[k,]))[1:10])
         cat(strrep('-',80), "\n")
     })

In [ ]:
## look at the reconstruction
Phat.gnmf <- lr.gnmf$L %*% lr.gnmf$R
summary(as.vector(Phat.gnmf))
ggplotm(pmin(Phat.gnmf, 0.0001), format="", show.axis=FALSE, mid="black")

# 2 PLSA

In [ ]:
## computing the 3-matrix decompositions
r <- 4
lr.gkl <- lee01.gkl(P, r, reps=5)
lsr.gkl <- nmf.lsr(lr.gkl) ## result as: L %*% S %*% R
slr.gkl <- nmf.slr(lr.gkl) ## result as: S %*% L %*% R

## LSR

In [ ]:
# Explore S
lsr.gkl$S
sum(lsr.gkl$S)

In [ ]:
# Explore R
with(lsr.gkl,
     for (k in 1:nrow(R)) {
         print(rev(sort(R[k,]))[1:10])
         cat(strrep('-',80), "\n")
     })

In [ ]:
# Explore L
dim(lsr.gkl$L)
lsr.gkl$L
sum(lsr.gkl$L[,1])
sum(lsr.gkl$L[,2])
sum(lsr.gkl$L[,3])
sum(lsr.gkl$L[,4])

## SLR

In [ ]:
# Explore L
slr.gkl$L

In [ ]:
# Explore R
slr.gkl$R
sum(slr.gkl$R[1,])
sum(slr.gkl$R[2,])
sum(slr.gkl$R[3,])
sum(slr.gkl$R[4,])
with(slr.gkl,
     for (k in 1:nrow(R)) {
         print(rev(sort(R[k,]))[1:10])
         cat(strrep('-',80), "\n")
     })

In [ ]:
# Explore S
with(slr.gkl,{
    O <- S %*% L %*% R
    S[ S < 0.0007045] <- 0
    C <- S %*% L %*% R
    print(norm(C - O, "F"))
    print(norm(P - O, "F"))
    print(norm(P - C, "F"))
})
sum(slr.gkl$S)
summary(slr.gkl$S)
slr.gkl$S

# 3 Clustering

In [ ]:
## true labels (DO NOT USE for clustering)
## 1=sci.crypt
## 2=sci.med
## 3=sci.space
## 4=soc.religion.christian
labels <- rep(1:4, each=100)

## to compute the confusion matrix between a clustering and the true labels, we
## first relabel every cluster so that cluster ids match labels to the extent
## possible. (Always do this.)
cm <- function(cluster.ids) {
    cluster.ids.matched <- match.labels(cluster.ids, labels)
    u = union(cluster.ids.matched, labels)
    t = table(factor(cluster.ids.matched, u), factor(labels, u))
    confusionMatrix(t)
}

## k-Means

In [ ]:
## example clustering (k-means with k=4)
cluster <- kmeans(P, 4, nstart=100)$cluster
cm(cluster)$overall["Accuracy"] ## just the accuracy of "cluster"
cm(cluster)                     ## confusion matrix and statistics of "cluster" (rows = predicted, columns = actual)

## k-Means on $\mathbf{U}_4\mathbf{\Sigma}_4$

In [ ]:
P.svd <- svd(P)
u.4 <- P.svd$u[,1:4]
d.4 <- diag(P.svd$d[1:4])

cluster <- kmeans(u.4 %*% d.4, 4, nstart=100)$cluster
cm(cluster)$overall["Accuracy"]
cm(cluster)

## k-Means on the $\mathbf{\tilde{L}}$ matrix of the NMF

In [ ]:
r <- 4
lr.gkl <- lee01.gkl(P, r, reps=5)

In [ ]:
cluster <- kmeans(lr.gkl$L, 4, nstart=100)$cluster
cm(cluster)$overall["Accuracy"]
cm(cluster)

## k-Means on the $\mathbf{{L'}}$ matrix of factorization $\mathbf{L'}\mathbf{\Sigma'}\mathbf{R'}$

In [ ]:
r <- 4
lr.gkl <- lee01.gkl(P, r, reps=5)
lsr.gkl <- nmf.lsr(lr.gkl) ## result as: L %*% S %*% R

In [ ]:
cluster <- kmeans(lsr.gkl$L, 4, nstart=100)$cluster
cm(cluster)$overall["Accuracy"]
cm(cluster)

## k-Means on the $\mathbf{{L''}}$ matrix of factorization $\mathbf{\Sigma''}\mathbf{L''}\mathbf{R''}$

In [ ]:
r <- 4
lr.gkl <- lee01.gkl(P, r, reps=5)
slr.gkl <- nmf.slr(lr.gkl) ## result as: L %*% S %*% R

In [ ]:
cluster <- kmeans(slr.gkl$L, 4, nstart=100)$cluster
cm(cluster)$overall["Accuracy"]
cm(cluster)